In [1]:
import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector

/opt/conda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [33]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    # zipfile读取文件后，得到文件名，再用f.read得到bytes。python3中bytes是对应编码的，这里应该是默认的utf-8
    # 再使用tensorflow的as_str进行解码后，切分为单词list。as_str将bytes或者unicode均解码为unicode，且能兼容不同的python版本
    # compat的意思就是compatible
#     print(type(f.read(f.namelist()[0])))
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

In [56]:
# 读取文件
filename = 'data/text8.zip'
vocabulary = read_data(filename)
print('Data size', len(vocabulary))

Data size 17005207


In [45]:
# 通过给定的词典大小，将出现次数较少的单词设置为UNK词，从而缩小词典
# 返回各个词在词典中的编号（codes），以及各词的词频等信息
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  # dictionary中存储每个单词到编号的映射
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  # data为存储所有单词编号的list
  data = list()
  # 将所有低频词的频率相加，作为UNK的频率
  unk_count = 0
  for word in words:
    index = dictionary.get(word, 0)
    if index == 0:  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  # 设置UNK的词频
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

In [57]:
# 生成词典和编号与词的映射
vocabulary_size = 50000
data, count, dictionary, reverse_dictionary = build_dataset(
    vocabulary, vocabulary_size)
# del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])



Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [59]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  # 为何batch_size一定要能整除num_skips?
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  # 生成一个大小为span的双端队列
  buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builtin
  # 当前数据索引 + 滑动窗口大小如果大于了data的长度，则从头开始
  if data_index + span > len(data):
    data_index = 0
  # 初始化双端队列，设置为data中的第一个窗口[0 ~ span]，不包括span, list的特性是大于等于第一个索引，小于最后一个索引  
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  # 总共需要生成batch_size个（contest，target)样本，分为batch_size // num_skips组，每组里面有num_skip个sample
  for i in range(batch_size // num_skips): # 取整
    # 由于 span一定是奇数，且一定以skip_window所在的位置为中心，所以可以以此获取context
    context_words = [w for w in range(span) if w != skip_window]
    # 从上下文词语中随机抽取num_skips个词语，用于生成一个batch中的一组sample
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      # batch里面的索引公式，提现了每组有nums_skips个sample，总共i组的推断
      # COBOW由contexts推断target，即包含多个(contexts, target)组成的sample，如：([the, brown], quick), ([quick, fox], brown)
      # 而skip-gram正好相反，由target预测某一个contex的概率，即：(quick, the), (quick, brown), (brown, quick), (brown, fox)
      # 这里batch存储的是target单测的索引，lables存储的是对应的context word的索引，每一对batch-labels为一个正样本
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]
    # 若滑动窗口到了数据的结尾，则双端队列的滑动窗口又从0开始
    if data_index == len(data):
      buffer.extend(data[0:span])
      data_index = span
    else: # 否则的话，双端队列对应的窗口往前移动一位
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  # 将data_index往前回溯span个位置，防止下次采样跳过上次batch中结尾的单词
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

In [61]:
data_index = 0
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
        reverse_dictionary[labels[i, 0]])
len(batch)


3081 originated -> 5234 anarchism
3081 originated -> 12 as
12 as -> 3081 originated
12 as -> 6 a
6 a -> 12 as
6 a -> 195 term
195 term -> 6 a
195 term -> 2 of


8

In [66]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

graph = tf.Graph()

with graph.as_default():

  # Input data.
  with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    with tf.name_scope('embeddings'):
      embeddings = tf.Variable(
          tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
      embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    with tf.name_scope('weights'):
      nce_weights = tf.Variable(
          tf.truncated_normal(
              [vocabulary_size, embedding_size],
              stddev=1.0 / math.sqrt(embedding_size)))
    with tf.name_scope('biases'):
      nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  # Explanation of the meaning of NCE loss:
  #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
  with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights=nce_weights,
            biases=nce_biases,
            labels=train_labels,
            inputs=embed,
            num_sampled=num_sampled,
            num_classes=vocabulary_size))

  # Add the loss value as a scalar to summary.
  tf.summary.scalar('loss', loss)

  # Construct the SGD optimizer using a learning rate of 1.0.
  with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                            valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Merge all summaries.
  merged = tf.summary.merge_all()

  # Add variable initializer.
  init = tf.global_variables_initializer()

  # Create a saver.
  saver = tf.train.Saver()

TypeError: reduce_sum() got an unexpected keyword argument 'keepdims'

 9017738.3G 5064M 2450M S 15.4  3.1 11h02:15 python train_ima 9018438.3G 5064M 2450M S 14.7  3.1 11h02:19 python train_ima21844.7 7h01:27244.7362104R 14.0 7h01:546944.019834R 14.011h02:47 5060M 2450M S 104.  3.1  6h59:15 python train_imaF1Help  F2Setup F3SearchF4FilterF5Tree  F6SortByF7Nice -F8Nice +F9Kill  F10Quit|||50.0%]   17 [|||||100.0%]   23 [       0.0%]6  [||||||50.0%]   12 [||||||50.0%]   18 [|||||100.0%]   24 [       0.0%]Mem[||||||||||||||||||||72.4G/157G]   Tasks: 17, 186 thr; 20 runningSwp[                         0K/0K]   Load average: 15.26 13.26 12.80 Uptime: 312 days(!), 14:15:29|  31.9||||||43|||   19|  26.1||||  29.6|| 33.6%||  26.9||40.6%|   20.8%|  30.1||||| 39.7|||   17.6|| 35.7|   25.5%|||   19.6|||||  30.5|| 44.1| 36.6%||||  28.9|  27.6|||43.5|  27.3%||||| 30.659314S 544.|||42.3|  23.87.433.8| 34.7  28.7||| 31.0|  36.018.729.6  33.6||  24.3|||42.0||| 35.0||  22.4| 33.10.4|  22.96.5739.06.6||  31.1612 running[24G 5069588484.9514G 506919.2 6h56:182204G 5069D 17.8

|44.05|   2130.05.0|49.3||  26.7|||40.4|   21.39.3||| 28.14.833.831.5||  24.1|2.9|||43.7|||4228.6|43.532.99.2|| 35.21168 13.35 12.84404G 5055773716.  2.1  0:52.514G 5055R 21.284G 5055S 17.72:0824G 5055S 17.01:38694G 5055S 17.01864G 50557.007684G 5055S 16.32:5494G 5055S 16.38:36754G 5055S 15.617734G 5055S 14.84:13|| 37.419.4||  21.39| 34.8|||51.8||| 34.8|  31.2||  25.7|| 35.235.7|   17.429| 36.2728.19.33.433.81.3|||43.87.9  30.92281861145974.02186818.51h02:072208R 17.02h23:0738R 4:132078 7h04:16688R 16.354748R 3:172038R  7h02:0288R 2:0828R 15.61:382.9|   20.3|   20.527.6|| 3641.15.2|||42.62.1|  31||  29.5||  24.67.9|| 32.419| 32.6|| 37.4|| 30.927.239.7|| 36.25.71.03033G 5064529862M 3536.  2.2  0:55.543G 5064S 15.48133G 50645.4 6h56:533G 50644.741683G 50644.711h02:5573G 50644.708963G 5064S 14.74h10:301723G 50644.011h01:3893G 5064S 14.01:2373G 5064S 14.02:17  33.8119.01|  32.1||| 32.08.3|| 41.1|   19.027.33.8|   22.7|||37.5|| 33.3|   200.5||  29.2|||42.39.7 39.9|||41.44.826.8144.47 13.20 

|  33.6||  32.7||  26.0|     6.2||  29.1  26  31.1|| 37.0|   15.0|47|80.3  30.8|| 35.2       0.0|  27.7|| 34.570.160.8  28.19| 42|   20.8|  21.9775165652.9503.  2.5  1:03.104657.96h59:0226560.90:491865S 42.57h01:250165S 15.17h09:3401686515.111h02:550065149:591776513.711h02:17565S 13.02:401826513.011h03:54| 35.029.2||||48.9|||  33.8||| 36.86|||42.43.1||  22.9| 42.5|| 34.95.00|||   15.6| 3|  25.7|| 38.7|  26.19.7|   21|   20.1||  21.45.08262725556262M 406164.61722131897218.311h08:3817771R 16.911h02:1820724.812h23:0887724.81:4019672114h10:318714.10917671R 14.111h01:2472R 14.12:21  31.9||| 29.550.829.168|  27.96.64.1|41.8  28.5| 36.7  30.5||||37.9||||||42.8|30.0834.5| 31.93.9||| 40.7|||41.5|   20.799448871878.  2.6  1:06.14548197:4204489.9 7h07:3589488.58:31248R 18.5 7h00:49684872:56487048R 171:4882486.43:543486.449|| 37.4  26.642.9| 37.9|43.17.3| 30.2||  25.73.2|| 33.6|   19.95.9| 38.2|  28.1|3.0| 29.52.629.8  31.71.3|| 35.5|| 33.82.23.0205 13.375546013117.  2.7  1:07.62054S 20.112h23:091

>5.98.438  29.2||||66.7||  22.12240.4||  25.44.8|  32.6|43.0|37.7%8||||  31.88.6|   22.82.9  28.0||  22.3|||39.6|| 33.88.7812494091.  2.5  1:13.55220226.42h23:10702R 17.41h01:50177271h02:126.092R 15.38:398623.20222.53:552062S 12.5 6h57:338122.51:51|||42.1740.7|| 40.3||||100.0%9.83.6| 36.9|   20.1|| 39.35.0|| 34.0||  25.9   1935|||| 34.7| 37.5||  29.1|   20.032.2|     9.0|  29.9|42.35.599363561975.  2.6  1:15.04196631.94h10:338363S 17.729163S 171:45212637.7 7h0063S 164022163S 16.3 6h56:2163S 15.6187635.611h01:417363S 15.64:168

In [64]:
!ls


Basic_word2vector.ipynb		 book
Recommander  System.xmind	 cf
Recommender System.ipynb	 data
__init__.py			 tensorFlow DeepFM实验.ipynb
algo.qq.com_641013010_testa.zip  util
